# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['0', 'k', 'f', 'm', 'x']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([21, 11, 23, 22, 24, 42, 30, 17,  2, 12])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[21, 11, 23, 22, 24, 42, 30, 17],
        [11, 23, 22, 24, 42, 30, 17,  2],
        [23, 22, 24, 42, 30, 17,  2, 12],
        [22, 24, 42, 30, 17,  2, 12, 24],
        [24, 42, 30, 17,  2, 12, 24,  3]])
y training data
tensor([[11, 23, 22, 24, 42, 30, 17,  2],
        [23, 22, 24, 42, 30, 17,  2, 12],
        [22, 24, 42, 30, 17,  2, 12, 24],
        [24, 42, 30, 17,  2, 12, 24,  3],
        [42, 30, 17,  2, 12, 24,  3, 18]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[-0.5037, -0.4606, -1.0284,  1.4244, -0.6913, -2.1196,  0.4278,
          -1.0677],
         [-0.2712, -0.2181,  2.1937,  0.5109,  0.0822,  0.1224, -0.7383,
          -2.5617],
         [ 0.7080,  0.7884, -0.1744, -0.4616,  1.0200,  1.2690,  0.5349,
           1.8285],
         [-0.5165,  1.6759,  1.0185, -1.1399, -0.1933,  0.6807, -0.3136,
           1.7759]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[ 0.3670, -0.5352, -0.5615,  1.2713,  0.7045, -2.3189,  1.6717,
          -0.8195],
         [ 0.0431,  0.8193, -0.6997,  0.1229,  1.8298, -0.6548,  1.1824,
           0.0806],
         [ 1.2184, -0.8640,  0.3499,  1.4709,  0.0571,  0.6448,  0.6276,
           0.3043],
         [ 0.8728, -0.2741,  0.2480,  0.7597, -1.7264,  0.1798, -0.4858,
           0.6839],
         [-1.5515, -0.4268,  0.4334,  0.2148, -1.8364,  0.0896,  1.1805,
           0.2158],
         [-0.8852, -0.6178, -0.5703, -0.4105,  0.3587,  0.3294,  0.0929,
          -0.2885],
         [-0.3182, -0.4844, -0.9233,  0.8547, -1.1227, -0.1346, -0.8138,
          -1.2769],
         [-0.1046, -1.3187, -0.6789, -0.6506,  0.0127, -0.3080, -0.6245,
          -0.0633]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [18]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [19]:
d_k = d_q = 4

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension


In [20]:
W_q.weight.shape

torch.Size([4, 8])

In [21]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [22]:
W_q.weight@x_embeddings_total[0][0]

tensor([ 0.2962,  0.4433, -0.4033,  0.0918], grad_fn=<MvBackward0>)

In [23]:
W_q(x_embeddings_total)

tensor([[[ 2.9622e-01,  4.4334e-01, -4.0333e-01,  9.1819e-02],
         [ 2.0940e-01, -1.9907e+00, -1.2677e+00, -2.5409e+00],
         [-4.4036e-01, -1.0597e+00, -9.9490e-01, -4.0069e-01],
         [ 3.5352e-02,  6.0534e-01,  7.5681e-02,  1.6958e+00],
         [ 2.1072e-01,  5.2485e-01,  5.1694e-01,  6.2034e-01],
         [-5.7203e-01,  8.1172e-01, -2.6101e-01, -6.2432e-01],
         [ 4.2714e-01,  1.7028e-01,  3.5651e-01,  6.0556e-01],
         [ 4.0804e-01, -3.5315e-01, -3.4155e-01,  6.7602e-01]],

        [[ 1.9107e-01,  4.4825e-03, -1.1271e+00, -8.4113e-01],
         [ 5.5712e-01, -2.5121e+00, -1.2270e+00, -1.6842e+00],
         [-7.1933e-01,  2.8078e-01, -6.8147e-01,  4.5110e-01],
         [-8.0519e-01,  9.6637e-01,  8.9220e-01,  1.6046e+00],
         [ 1.3123e+00,  2.8796e-01,  6.3086e-04, -3.2865e-01],
         [-5.9724e-01,  1.2756e+00, -2.9307e-01, -3.9490e-01],
         [ 1.0126e+00,  1.7249e-01,  5.3831e-01,  1.0307e+00],
         [-3.2659e-01, -2.1117e+00, -1.6715e+00, -3.2

In [24]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [25]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 4]) torch.Size([5, 8, 4]) torch.Size([5, 8, 8])


In [26]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [27]:
print(Q[0][:3])
print(K[0][:3])

tensor([[ 0.2962,  0.4433, -0.4033,  0.0918],
        [ 0.2094, -1.9907, -1.2677, -2.5409],
        [-0.4404, -1.0597, -0.9949, -0.4007]], grad_fn=<SliceBackward0>)
tensor([[ 0.4785, -1.8924,  1.0503,  0.9186],
        [ 1.7903,  0.0951,  0.1838, -0.3514],
        [-0.3516,  0.0406,  0.7802,  1.3018]], grad_fn=<SliceBackward0>)



well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present


ok let's say no linear layer was present, than the Q*K.T would just give me the cosine similarity between static embeddings of tokens

let's say the attention matrix is

`A`

here `A[i][j]` would be the cosine similarity between the ith token and jth token  (here when I say token I mean the embedding of token)

now let's see what would `A@V` do, here I am assuming `V` to is just the static emebdding


`A_softmax` -> is softmax function applied on `A` across each column (why column? cause weights are distributed across column see below)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

here `V[i]` represents the static embedding for the ith word, so `V[i]` is a vector, and ``V`[i]`` is the final vector after attention matrix multiplication

``V`[i]`` we can think of it as change or how the original word gets modified

so when taking the weighted sum it tells us, in direction of which static embedding should the input embedding move to the most and least (depending on the weight), so that it represents that modified vector would represent the meaning of the entire sentence, **now I see the poblem with just using static embedding, the problem is if we just use static embedding to compute cosine similarity it would nudge the input emebedding to this layer in the direction which is most similar to the word which we see in the sentence, which we don't always want**

side note: one more thing I learned is that these query and key vector live in a space which is smaller than the static embedding dimension, so it would help computationaly too

In [28]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])

In [29]:
attention_matrix[0]

tensor([[-1.0365,  0.4661, -0.2813,  0.0463, -0.1883, -0.2296, -0.4358, -0.2512],
        [ 0.2019,  0.8453, -4.4513, -1.1401,  1.3999,  0.5916, -0.0298,  0.4806],
        [ 0.3817, -0.9313, -1.1860,  0.9979,  0.5567,  0.3707,  0.1498,  0.8662],
        [ 0.5086, -0.4611,  2.2789,  0.5995, -0.6054,  0.3011, -0.1403,  0.1271],
        [ 0.2204,  0.3042,  1.1581, -0.4140, -0.3638,  0.0236, -0.0526, -0.3158],
        [-2.6575, -0.7755, -0.7823,  1.7996, -0.2031, -1.4972, -0.0844, -0.2259],
        [ 0.8128,  0.6337,  0.9232, -0.9524, -0.2063,  0.4751, -0.1114, -0.2157],
        [ 1.1258,  0.3966,  0.4558, -0.6336,  0.0213,  0.8962, -0.2089,  0.2331]],
       grad_fn=<SelectBackward0>)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

now in self attention when we compute the change required for ith vector we don't want any token above ith token to be deciding how it changes, cause during inference we don't have that information at all so we would need to find a way so that above becomes

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ... + V[i]*A_softmax[i][i]

```

so to do this we convert the lower triangular part equal infinity so that when we do softmax we get zero

In [30]:
print(x_batch[0])
print(y_batch[0])

tensor([21, 11, 23, 22, 24, 42, 30, 17])
tensor([11, 23, 22, 24, 42, 30, 17,  2])


In [31]:
torch.ones_like(attention_matrix[0])

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [32]:
torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool() # should the diagonal also have no contribution?

tensor([[False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [33]:
attention_matrix[0].masked_fill(torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool(), float('-inf'))

tensor([[-1.0365,  0.4661, -0.2813,  0.0463, -0.1883, -0.2296, -0.4358, -0.2512],
        [   -inf,  0.8453, -4.4513, -1.1401,  1.3999,  0.5916, -0.0298,  0.4806],
        [   -inf,    -inf, -1.1860,  0.9979,  0.5567,  0.3707,  0.1498,  0.8662],
        [   -inf,    -inf,    -inf,  0.5995, -0.6054,  0.3011, -0.1403,  0.1271],
        [   -inf,    -inf,    -inf,    -inf, -0.3638,  0.0236, -0.0526, -0.3158],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -1.4972, -0.0844, -0.2259],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1114, -0.2157],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.2331]],
       grad_fn=<MaskedFillBackward0>)

In [34]:
attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))[0]

tensor([[-1.0365,  0.4661, -0.2813,  0.0463, -0.1883, -0.2296, -0.4358, -0.2512],
        [   -inf,  0.8453, -4.4513, -1.1401,  1.3999,  0.5916, -0.0298,  0.4806],
        [   -inf,    -inf, -1.1860,  0.9979,  0.5567,  0.3707,  0.1498,  0.8662],
        [   -inf,    -inf,    -inf,  0.5995, -0.6054,  0.3011, -0.1403,  0.1271],
        [   -inf,    -inf,    -inf,    -inf, -0.3638,  0.0236, -0.0526, -0.3158],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -1.4972, -0.0844, -0.2259],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1114, -0.2157],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.2331]],
       grad_fn=<SelectBackward0>)

In [35]:
masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

In [36]:
masked_attention_matrix[0]

tensor([[-1.0365,  0.4661, -0.2813,  0.0463, -0.1883, -0.2296, -0.4358, -0.2512],
        [   -inf,  0.8453, -4.4513, -1.1401,  1.3999,  0.5916, -0.0298,  0.4806],
        [   -inf,    -inf, -1.1860,  0.9979,  0.5567,  0.3707,  0.1498,  0.8662],
        [   -inf,    -inf,    -inf,  0.5995, -0.6054,  0.3011, -0.1403,  0.1271],
        [   -inf,    -inf,    -inf,    -inf, -0.3638,  0.0236, -0.0526, -0.3158],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -1.4972, -0.0844, -0.2259],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1114, -0.2157],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.2331]],
       grad_fn=<SelectBackward0>)

In [37]:
import torch.nn.functional as F


F.softmax(masked_attention_matrix[0], dim=-2)

tensor([[1.0000, 0.4063, 0.7042, 0.1775, 0.1053, 0.1195, 0.1009, 0.0819],
        [0.0000, 0.5937, 0.0109, 0.0542, 0.5153, 0.2717, 0.1514, 0.1702],
        [0.0000, 0.0000, 0.2850, 0.4597, 0.2217, 0.2179, 0.1812, 0.2502],
        [0.0000, 0.0000, 0.0000, 0.3086, 0.0694, 0.2032, 0.1356, 0.1195],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0883, 0.1540, 0.1480, 0.0767],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0337, 0.1434, 0.0839],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1396, 0.0848],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1328]],
       grad_fn=<SoftmaxBackward0>)

### should the diagonal (i.e the current token) have any contribution towards update of input embedding vector?

if it does then ``V`[0] = V[0]`` and that would mean the updated embedding would become ``E_updated[0] = V[0] + E[0]`` need to think more about it

but according to my understanding `V[0]` is same as the absolute embedding of the token so `V[0] + E[0]` would be pracitically `2*absolute_mebdding[0]` since even `E[0]` was created any info about token after that so just itself, but since we don layer norm `norm(2*absolute_embedding[0]) = absolute_embedding[0]` so even if it scales a lot layer normalization brings down the scale of embedding vector so it fits with rest of embedding vector's scale

In [38]:
batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

In [39]:
batch_attention_matrix[0]

tensor([[1.0000, 0.4063, 0.7042, 0.1775, 0.1053, 0.1195, 0.1009, 0.0819],
        [0.0000, 0.5937, 0.0109, 0.0542, 0.5153, 0.2717, 0.1514, 0.1702],
        [0.0000, 0.0000, 0.2850, 0.4597, 0.2217, 0.2179, 0.1812, 0.2502],
        [0.0000, 0.0000, 0.0000, 0.3086, 0.0694, 0.2032, 0.1356, 0.1195],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0883, 0.1540, 0.1480, 0.0767],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0337, 0.1434, 0.0839],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1396, 0.0848],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1328]],
       grad_fn=<SelectBackward0>)

In [40]:
V[0]

tensor([[ 2.8822e+00,  8.1252e-01, -1.2195e+00,  1.9457e+00,  2.7619e-01,
          7.7019e-01, -8.2986e-01, -1.9775e+00],
        [ 3.0802e-01,  3.9843e-01, -1.0687e+00, -1.5616e+00,  7.8532e-01,
         -9.1509e-01,  3.0454e-01,  5.3056e-02],
        [ 1.5264e+00, -4.9802e-01, -2.9694e-02,  1.8806e-01,  1.7161e-01,
          7.8767e-01, -9.2485e-01, -1.6506e+00],
        [-8.1766e-01, -9.5586e-01,  1.2630e+00,  2.1786e-01, -5.7029e-01,
          2.1527e-01, -2.3363e-01,  6.6448e-03],
        [ 1.3550e-01,  3.1043e-01, -1.2017e-01,  5.3248e-01, -1.3246e-01,
          2.1973e-03, -1.5388e-01, -4.8836e-02],
        [-1.3120e-02,  7.2783e-01,  2.8863e-01,  1.3408e+00,  1.3093e+00,
          1.7083e-01,  1.6660e-01, -7.6276e-01],
        [ 1.0175e+00,  1.3398e-01,  4.7444e-01,  1.1533e+00, -8.3743e-01,
          1.8518e+00, -4.0915e-01, -7.5429e-01],
        [ 7.8381e-01,  7.6704e-02,  2.6689e-01,  7.3089e-01,  1.5441e-01,
          2.1513e-01, -9.8640e-01, -1.4326e+00]], grad_fn=<Select

In [41]:
batch_attention_matrix[0]@V[0]

tensor([[ 4.1165,  0.5935, -1.3589,  1.8748,  0.6856,  1.2163, -1.5171, -3.4067],
        [ 0.5088,  0.5704, -0.4326,  0.0245,  0.6242, -0.1585, -0.1057, -0.5765],
        [ 0.4667, -0.3104,  0.7611,  0.9558, -0.0705,  0.7505, -0.6897, -1.1394],
        [-0.0140, -0.0982,  0.5363,  0.6203, -0.0142,  0.3781, -0.2222, -0.4298],
        [ 0.2207,  0.1652,  0.1245,  0.4802,  0.0778,  0.3171, -0.1242, -0.3433],
        [ 0.2112,  0.0501,  0.1001,  0.2718, -0.0630,  0.2893, -0.1359, -0.2541],
        [ 0.2085,  0.0252,  0.0888,  0.2229, -0.1038,  0.2767, -0.1407, -0.2268],
        [ 0.1041,  0.0102,  0.0355,  0.0971,  0.0205,  0.0286, -0.1310, -0.1903]],
       grad_fn=<MmBackward0>)

In [42]:
V_prime = batch_attention_matrix@V

In [43]:
embedding_unnormalized = V_prime + x_embeddings_total

# Layer normalization

we just normalize across the embedding dimension to make sure this addition does not scale the embedding vector to much across each layer so that it has the scale of the original embedding vector

In [44]:
mean_emebdding = embedding_unnormalized.mean(dim=-1)
mean_emebdding.shape

torch.Size([5, 8])

In [45]:
std_embedding = embedding_unnormalized.std(dim=-1)
std_embedding.shape

torch.Size([5, 8])

In [46]:
embedding_unnormalized.shape

torch.Size([5, 8, 8])

In [47]:
print(mean_emebdding[0][0], std_embedding[0][0])

tensor(-0.2358, grad_fn=<SelectBackward0>) tensor(3.4888, grad_fn=<SelectBackward0>)


In [48]:
(embedding_unnormalized[0][0] - mean_emebdding[0][0])/(std_embedding[0][0] + 0.0001)

tensor([ 1.2030, -0.4839, -1.0330,  1.6250,  0.3421, -0.6380,  0.0268, -1.0420],
       grad_fn=<DivBackward0>)

In [49]:
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast
embedding_normalized[0][0]

tensor([ 1.2030, -0.4839, -1.0330,  1.6250,  0.3421, -0.6380,  0.0268, -1.0420],
       grad_fn=<SelectBackward0>)

# Multi head self attention

![](20251130180338.png)

Functionaly what multi headed attention does is, it employs multiple smaller self attention blocks, the input to each is the full embedding vectors, and the output are vectors with a smaller dimension


```

let's say the input is [6, 6, 6]  (here 6 represents the dimension of the embedding vector which goes in, and 3 is the context length)


[6, 6, 6] -> head 1 -> [2, 2, 2]'1 (here I am just using '1 as a label)

[6, 6, 6] -> head 2 -> [2, 2, 2]'2

[6, 6, 6] -> head 3 -> [2, 2, 2]'3


concat([2, 2, 2]'1, [2, 2, 2]'2, [2, 2, 2]'3) -> [6, 6, 6]'

[6, 6, 6]' -> linear layer -> [6, 6, 6]'' --> residual connection --> [6, 6, 6]'' + [6, 6, 6] --> layer norm --> [6, 6, 6]final

```



In [50]:
# writing everything we have done in one block

# ----multi head repetition--------
d_k = d_q = 4
d_v = EMBEDDING_DIMENSION

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

V_prime = batch_attention_matrix@V
# ----multi head repetition--------

embedding_unnormalized = V_prime + x_embeddings_total

mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)

embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


In [51]:
# let me create a class out of the repeated block

class SingleHead():

    def __init__(self, d_k, d_q, d_v, input_dimension) -> None:
        
        self.W_q = nn.Linear(input_dimension, d_q, bias = False)
        self.W_k = nn.Linear(input_dimension, d_k, bias = False)
        self.W_v = nn.Linear(input_dimension, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension
        pass

    def forward(self, input_embeddings):
        Q, K, V = self.W_q(input_embeddings), self.W_k(input_embeddings), self.W_v(input_embeddings)

        attention_matrix = Q@K.transpose(-2, -1) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

        masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

        batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

        V_prime = batch_attention_matrix@V

        return V_prime

        
    

In [52]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [53]:
# since our embedding is of size 8 I am gonna create 2 heads with output dimensino 4

head1, head2 = SingleHead(2, 2, 4, 8), SingleHead(2, 2, 4, 8)

In [54]:
x_out_head_1 = head1.forward(x_embeddings_total)[0]
print(x_out_head_1)
x_out_head_2 = head2.forward(x_embeddings_total)[0]
print(x_out_head_2)
print(torch.cat([x_out_head_1, x_out_head_2], dim=-1))

tensor([[-0.0737, -1.9968,  0.8206,  0.1649],
        [-0.4561, -0.4630, -0.3824,  0.5564],
        [-0.1686, -0.4715,  0.3744,  0.5112],
        [-0.2452, -0.2064,  0.8486,  0.0039],
        [-0.1601, -0.2369,  0.4407, -0.1236],
        [ 0.0022, -0.2633,  0.2046, -0.0096],
        [ 0.0198, -0.1867,  0.1099, -0.0108],
        [-0.1292, -0.1143,  0.0695, -0.0233]], grad_fn=<SelectBackward0>)
tensor([[-1.9296, -0.2495,  0.5440,  1.8510],
        [-0.3440,  0.6963,  1.1398,  0.5546],
        [ 0.6122,  0.3662,  0.8504, -0.0491],
        [-0.0973,  1.1241,  0.6411,  1.3044],
        [ 0.0126,  0.2975,  0.1584,  0.3997],
        [-0.0123,  0.2337,  0.1595,  0.2554],
        [ 0.0282,  0.0328,  0.0433,  0.0269],
        [ 0.0446,  0.0544,  0.0691,  0.0485]], grad_fn=<SelectBackward0>)
tensor([[-0.0737, -1.9968,  0.8206,  0.1649, -1.9296, -0.2495,  0.5440,  1.8510],
        [-0.4561, -0.4630, -0.3824,  0.5564, -0.3440,  0.6963,  1.1398,  0.5546],
        [-0.1686, -0.4715,  0.3744,  0.5112,

In [55]:
x_out_embedding_total = torch.cat([head1.forward(x_embeddings_total), head2.forward(x_embeddings_total)], dim= -1)
x_out_embedding_total[0]

tensor([[-0.0737, -1.9968,  0.8206,  0.1649, -1.9296, -0.2495,  0.5440,  1.8510],
        [-0.4561, -0.4630, -0.3824,  0.5564, -0.3440,  0.6963,  1.1398,  0.5546],
        [-0.1686, -0.4715,  0.3744,  0.5112,  0.6122,  0.3662,  0.8504, -0.0491],
        [-0.2452, -0.2064,  0.8486,  0.0039, -0.0973,  1.1241,  0.6411,  1.3044],
        [-0.1601, -0.2369,  0.4407, -0.1236,  0.0126,  0.2975,  0.1584,  0.3997],
        [ 0.0022, -0.2633,  0.2046, -0.0096, -0.0123,  0.2337,  0.1595,  0.2554],
        [ 0.0198, -0.1867,  0.1099, -0.0108,  0.0282,  0.0328,  0.0433,  0.0269],
        [-0.1292, -0.1143,  0.0695, -0.0233,  0.0446,  0.0544,  0.0691,  0.0485]],
       grad_fn=<SelectBackward0>)

In [56]:
# add and norm 

embedding_unnormalized = x_out_embedding_total + x_embeddings_total
mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


Now that I have shown functionaly how multi head attention works we need to understand what it really means

it is really like head 1 decides from 0 to 3rd direction which direction it should move in and head 2 decide from 4th to 7th which direction embedding should move in

why we concat here ? why we not add them up? this I did not understand conceptually when I think of multi heads adding up, then I could argue that each head might have its own interpretation of how the space looks like which would might cause contradictory changes to the embeddings, another way to think of it is that adding all of them up would lead to a explotion in the value for each embedding dimension, tho layer norm should fix that

this still needs to be debated, I will have to think more deeply about what it means to add them up vs concat them

ok so it turns out I may have missed the feed forward step which has to be supposedly after every multi head attention layer

So the flow should be

1. Multi-Head self attention
2. Add residual
3. Layer norm
4. feed forward
5. add residual
6. layer norm

in my above example I missed from 3, so let me add that


In [57]:
linear_layer = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION)

embedding_mixed = linear_layer(embedding_normalized) # embedding_normalized vector supposedly have all the information in bits and piece 0 to 3rd dim comes from head 1 3rd to 6th from another and so on, 
# the embedding_mixed is supposed to have the mixed info about all these layers
embedding_mixed = embedding_normalized + embedding_mixed # residual connection
mean_embedding_mixed = embedding_mixed.mean(dim=-1)
std_embedding_mixed = embedding_mixed.std(dim=-1)

embedding_mixed_normalized = (embedding_mixed - mean_embedding_mixed.unsqueeze(-1))/(std_embedding_mixed.unsqueeze(-1) + 0.0001)

In [58]:
embedding_mixed_normalized[0]

tensor([[-0.2223, -0.6036, -0.2103,  1.7107, -0.2706, -1.6552,  0.7229,  0.5283],
        [-0.7689,  0.3730, -1.0578, -0.0193,  1.3909,  0.3113,  1.1285, -1.3577],
        [-0.7461, -0.1377, -0.5951,  1.6825, -0.3437, -1.0435,  1.4075, -0.2239],
        [ 0.3722, -0.7045, -0.2144, -0.9540, -0.6403,  1.3113, -0.7962,  1.6260],
        [-0.4414, -1.1687,  1.4907,  0.1020,  0.7014, -1.0049, -0.7485,  1.0693],
        [-1.8770, -0.1055,  0.0631,  0.4470,  1.2735, -0.7064, -0.1516,  1.0569],
        [-0.0672, -0.5733, -0.3826,  1.9516, -0.6731, -0.3870,  1.0903, -0.9586],
        [-0.2752, -1.6222, -0.0931,  0.7374,  0.4912, -0.8739, -0.0175,  1.6533]],
       grad_fn=<SelectBackward0>)

Ok now this constitutes of the full multihead attention block, I still feel like the explanation:
"Each head computes its own bits of feature that it understands, and then the feed forward layer after that combines and picks and mixes the relevant info to form a refined vector" still a bit hand wavy I might need to actually see what this layer is doing, for that I will have to analyze a already existing pretrained model, which would be the next thing I will do, since I don't have money to buy GPUs I will have to rely on analyzing already trained models, let's continue with next stuff

In [59]:
# project the existing embedding vectors into existing vocab space

project_to_vocab_layer = nn.Linear(EMBEDDING_DIMENSION, len(vocabulary_list))

logits = project_to_vocab_layer(embedding_mixed_normalized)

In [60]:
logits[0].shape # context length, vocab size

torch.Size([8, 44])

In [61]:
# exploding logits and expected value for easy cross entropy computation

logits_exploded = logits.view(logits.shape[0]*logits.shape[1], logits.shape[2])


In [62]:
targets_exploded = y_batch.view(y_batch.shape[0]*y_batch.shape[1])

In [63]:
F.cross_entropy(logits_exploded,targets_exploded)

tensor(3.7880, grad_fn=<NllLossBackward0>)

finally completed one single pass through transformers layers, next I will create modules out of it